---
---

# Team M.V.P Data Cleaning Submission

---
---

Submitted by:

 Vivek Venkateshprasad \
 862468868 \
 vv002

 Prathik Somanath \
 862467832 \
 psoma005
 
 Maedeh Makki \
 862393635 \
 mmakk004

 ---
 ---

---

### Step 1: Downloading the dataset and performing Basic Cleaning

---


We download the accident dataset.

Only problem is ... Our accident dataset is Huuuuge, 7.7 Million Records Huge. None of our local computer could run this. So, we decided to use a smaller version of our dataset.

https://huggingface.co/datasets/nateraw/us-accidents


In [55]:
import pandas as pd

df = pd.read_csv("accidents.csv")
df.head(5)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


We see that there are a lot columns which are not useful to us. We need to clean this data

Let us take a look at all the columns and determine which is useful to us

In [56]:
df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [57]:
useful_columns = ['State', 'Severity', 'Start_Time','Start_Lat','Start_Lng']
df = df[useful_columns]
df.head(5)

,State,Severity,Start_Time,Start_Lat,Start_Lng
0,OH,3,2016-02-08 00:37:08,40.108910,-83.092860
1,OH,2,2016-02-08 05:56:20,39.865420,-84.062800
2,OH,2,2016-02-08 06:15:39,39.102660,-84.524680
3,OH,2,2016-02-08 06:51:45,41.062130,-81.537840
4,OH,3,2016-02-08 07:53:43,39.172393,-84.492792


Okay, that looks good. Now, we need to merge this with the accident dataset. Let us load that up from here:

https://www.gigasheet.com/sample-data/per-capita-us-alcohol-consumption

In [58]:
df2 = pd.read_csv("alcohol.csv")
df2.head()

,Year,State,Beverage_Type,Gallons_of_Beverage,Gallons_of_Ethanol,Population_14_Plus,Ethanol_Per_Capita_14_Plus,Decile_14_Plus,Population_21_Plus,Ethanol_Per_Capita_21_Plus,Decile_21_Plus,Data_Source_Type,Time_Varying_ABV,Gallons_of_Ethanol_from_ABV
0,1970,Alabama,Spirits,3863000.0,1738350,249900,6956,9.0,202000,8606,9.0,.,NaN,NaN
1,1970,Alabama,Wine,1412000.0,225920,249900,904,9.0,202000,1118,9.0,.,NaN,NaN
2,1970,Alabama,Beer,33098000.0,1489410,249900,5960,10.0,202000,7373,10.0,.,NaN,NaN
3,1970,Alabama,All Beverages,NaN,3453680,249900,13820,10.0,202000,17097,10.0,.,NaN,NaN
4,1970,Alaska,Spirits,945000.0,425250,20500,20744,1.0,16500,25773,1.0,.,NaN,NaN


Let us look at the available columns in this dataset

In [59]:
df2.columns

Index(['Year', 'State', 'Beverage_Type', 'Gallons_of_Beverage',
       'Gallons_of_Ethanol', 'Population_14_Plus',
       'Ethanol_Per_Capita_14_Plus', 'Decile_14_Plus', 'Population_21_Plus',
       'Ethanol_Per_Capita_21_Plus', 'Decile_21_Plus', 'Data_Source_Type',
       'Time_Varying_ABV', 'Gallons_of_Ethanol_from_ABV'],
      dtype='object')

We don't have any age groups mentioned in our dataset, so we need to remove it. We also do not care about how many beverages a person had. We just care about how drunk they got and that can be extracted from the "Gallons_of_ethanol" column. We also need to merge the populations and get the total population to get a new ethanol_per_capita column.

In [60]:
useful_columns = ['Year', 'State', 'Beverage_Type','Gallons_of_Ethanol', 'Population_14_Plus', 'Population_21_Plus']
df2 = df2[useful_columns]
df2.head(5)

,Year,State,Beverage_Type,Gallons_of_Ethanol,Population_14_Plus,Population_21_Plus
0,1970,Alabama,Spirits,1738350,249900,202000
1,1970,Alabama,Wine,225920,249900,202000
2,1970,Alabama,Beer,1489410,249900,202000
3,1970,Alabama,All Beverages,3453680,249900,202000
4,1970,Alaska,Spirits,425250,20500,16500


We can immediately see a problem here. There is a semantic mismatch in the state names from the first column to the second.

For example, 

df1["State"] = "OH"

df2["State"] = "Ohio"

Let us fix this

I found this mapping dictionary here: https://gist.github.com/mshafrir/2646763

In [61]:
state_abbreviation_to_full_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
    'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
    'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon',
    'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
    'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

In [62]:
df["State"] = df["State"].replace(state_abbreviation_to_full_name)
df.head(5)

,State,Severity,Start_Time,Start_Lat,Start_Lng
0,Ohio,3,2016-02-08 00:37:08,40.108910,-83.092860
1,Ohio,2,2016-02-08 05:56:20,39.865420,-84.062800
2,Ohio,2,2016-02-08 06:15:39,39.102660,-84.524680
3,Ohio,2,2016-02-08 06:51:45,41.062130,-81.537840
4,Ohio,3,2016-02-08 07:53:43,39.172393,-84.492792


Looks good. Now we need to add the popualtions of the two columns and create a new column just called "population" in the alcohol dataset. Then calculate the new ethanol per capita

In [63]:
df2['Population'] = df2["Population_14_Plus"] + df2["Population_21_Plus"]
df2['Ethanol_per_capita'] = round(df2['Gallons_of_Ethanol'] / df2['Population'] * 1000)
df2 = df2.drop(['Population_14_Plus', 'Population_21_Plus'], axis=1)
df2.head(5)

,Year,State,Beverage_Type,Gallons_of_Ethanol,Population,Ethanol_per_capita
0,1970,Alabama,Spirits,1738350,451900,3847.0
1,1970,Alabama,Wine,225920,451900,500.0
2,1970,Alabama,Beer,1489410,451900,3296.0
3,1970,Alabama,All Beverages,3453680,451900,7643.0
4,1970,Alaska,Spirits,425250,37000,11493.0


Okay, Now our accident dataset only is present from 2016 - 2021. So, all the data previous and after it is useless to us. Let us clean this out

In [64]:
df2 = df2[(df2['Year'] > 2015) & (df2['Year'] < 2022)].reset_index(drop=True)
df2.head(5)

,Year,State,Beverage_Type,Gallons_of_Ethanol,Population,Ethanol_per_capita
0,2016,Alabama,Spirits,2631168,760007,3462.0
1,2016,Alabama,Wine,1043962,760007,1374.0
2,2016,Alabama,Beer,4401225,760007,5791.0
3,2016,Alabama,All Beverages,8076355,760007,10627.0
4,2016,Alaska,Spirits,732610,112306,6523.0


In [65]:
df2["State"].value_counts().shape

(56,)

This doesn't seem right. Last time I checked there were 50 States + D.C. So why are there 56 entries ?

On closer inspection we see some data for 'Northeast Region', 'Midwest Region', 'South Region', 'West Region', 'United States' . We can drop these

In [66]:
states_to_drop = ['Northeast Region', 'Midwest Region', 'South Region', 'West Region', 'United States']

df2 = df2[~df2['State'].isin(states_to_drop)]
df2["State"].value_counts().shape

(51,)

This looks better.

This marks and end of our basic cleaning

---

### Step 2:
### Processing Data for visualizations and further analysis

---




#### Processing Accident Data by State and Year
This code snippet demonstrates how to process the accident data by converting date information, handling missing values, extracting year, and aggregating accident counts by state and year.

First, let us first fix the datetime format in our accident dataset

In [67]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='mixed') # it has multiple datetime formats hence mixed

# Create new columns for day of the week, date, and year
df['Day_of_Week'] = df['Start_Time'].dt.day_name()      
df['Date'] = df['Start_Time'].dt.date                
df['Year'] = df['Start_Time'].dt.year 

df.head(2)

,State,Severity,Start_Time,Start_Lat,Start_Lng,Day_of_Week,Date,Year
0,Ohio,3,2016-02-08 00:37:08,40.10891,-83.09286,Monday,2016-02-08,2016
1,Ohio,2,2016-02-08 05:56:20,39.86542,-84.06280,Monday,2016-02-08,2016


Let us now count the accidents and group them by the state and year

In [68]:
accidents_by_state_and_year = df.groupby(["State", "Year"]).size().reset_index(name="Accident_Count")
accidents_by_state_and_year.head(6)

,State,Year,Accident_Count
0,Alabama,2016,133
1,Alabama,2017,600
2,Alabama,2018,935
3,Alabama,2019,1036
4,Alabama,2020,5016
5,Alabama,2021,11602


#### Merging Alcohol Consumption and Accident Data by State and Year

This code snippet merges two datasets, one on alcohol consumption and the other on accident counts, by state and year. It then filters out regional aggregate rows, focusing on specific states.

In [69]:
merged_df = pd.merge(df2, accidents_by_state_and_year, on=['State', 'Year'], how='left').dropna()
merged_df.head(5)


,Year,State,Beverage_Type,Gallons_of_Ethanol,Population,Ethanol_per_capita,Accident_Count
0,2016,Alabama,Spirits,2631168,760007,3462.0,133.0
1,2016,Alabama,Wine,1043962,760007,1374.0,133.0
2,2016,Alabama,Beer,4401225,760007,5791.0,133.0
3,2016,Alabama,All Beverages,8076355,760007,10627.0,133.0
8,2016,Arizona,Spirits,4473105,1070605,4178.0,2715.0


In [70]:
#Let us take the data for all the beverages.
merged_df_all_beverages = merged_df[merged_df['Beverage_Type'] == 'All Beverages']
merged_df_all_beverages.head(5)

,Year,State,Beverage_Type,Gallons_of_Ethanol,Population,Ethanol_per_capita,Accident_Count
3,2016,Alabama,All Beverages,8076355,760007,10627.0,133.0
11,2016,Arizona,All Beverages,13189089,1070605,12319.0,2715.0
15,2016,Arkansas,All Beverages,4392473,461171,9525.0,26.0
19,2016,California,All Beverages,75148499,6060076,12401.0,33592.0
23,2016,Colorado,All Beverages,12837432,862980,14876.0,1286.0


Okay that's good. Way to go Arkansas you have very less accidents. However, we need to find out the accident per capita to get the true picture. Lets create a column for that

In [ ]:
merged_df_all_beverages['Accidents_per_capita'] = round(merged_df_all_beverages['Accident_Count'] / merged_df_all_beverages['Population'] * 10000)
merged_df_all_beverages.head(5)

C:\Users\Vv27\AppData\Local\Temp\ipykernel_1956\2511412203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_all_beverages['Accidents_per_capita'] = round(merged_df_all_beverages['Accident_Count'] / merged_df_all_beverages['Population']*10000)


,Year,State,Beverage_Type,Gallons_of_Ethanol,Population,Ethanol_per_capita,Accident_Count,Accidents_per_capita
3,2016,Alabama,All Beverages,8076355,760007,10627.0,133.0,2.0
11,2016,Arizona,All Beverages,13189089,1070605,12319.0,2715.0,25.0
15,2016,Arkansas,All Beverages,4392473,461171,9525.0,26.0,1.0
19,2016,California,All Beverages,75148499,6060076,12401.0,33592.0,55.0
23,2016,Colorado,All Beverages,12837432,862980,14876.0,1286.0,15.0


Had to add a multiplier of 10000 because some states had way too few accidents compared to people. Still Arkansas looks to be doing well. Go Arkansas !

#### Holiday trend analysis 
Now we need to add more columns to our dataframe to analyze accident trends. Whether they have occured during holidays, which day of the week, etc.. 

In [75]:
import holidays
#this package will provide us the list of dates of holidays
for date, name in sorted(holidays.US(years=2024).items()):
    print(date, name)

2024-01-01 New Year's Day
2024-01-15 Martin Luther King Jr. Day
2024-02-19 Washington's Birthday
2024-05-27 Memorial Day
2024-06-19 Juneteenth National Independence Day
2024-07-04 Independence Day
2024-09-02 Labor Day
2024-10-14 Columbus Day
2024-11-11 Veterans Day
2024-11-28 Thanksgiving
2024-12-25 Christmas Day


In [76]:
# List of US holidays
us_holidays = holidays.US()

# Create Holiday flag and Holiday name columns
df['Is_Holiday'] = df['Start_Time'].apply(lambda x: x in us_holidays)
df['Holiday_Name'] = df['Start_Time'].apply(lambda x: us_holidays.get(x) if x in us_holidays else None)
df.head(5)

,State,Severity,Start_Time,Start_Lat,Start_Lng,Day_of_Week,Date,Year,Is_Holiday,Holiday_Name
0,Ohio,3,2016-02-08 00:37:08,40.108910,-83.092860,Monday,2016-02-08,2016,False,None
1,Ohio,2,2016-02-08 05:56:20,39.865420,-84.062800,Monday,2016-02-08,2016,False,None
2,Ohio,2,2016-02-08 06:15:39,39.102660,-84.524680,Monday,2016-02-08,2016,False,None
3,Ohio,2,2016-02-08 06:51:45,41.062130,-81.537840,Monday,2016-02-08,2016,False,None
4,Ohio,3,2016-02-08 07:53:43,39.172393,-84.492792,Monday,2016-02-08,2016,False,None


Lets check out what accidents are happening on Christmas

In [81]:
holiday_df = df[df['Holiday_Name'] == 'Christmas Day']
holiday_df.head(5)

,State,Severity,Start_Time,Start_Lat,Start_Lng,Day_of_Week,Date,Year,Is_Holiday,Holiday_Name
14393,Minnesota,2,2016-12-25 01:51:48,44.993560,-92.960070,Sunday,2016-12-25,2016,True,Christmas Day
14394,Texas,3,2016-12-25 02:09:33,32.767159,-96.836603,Sunday,2016-12-25,2016,True,Christmas Day
14395,Minnesota,2,2016-12-25 02:00:43,43.767220,-93.312750,Sunday,2016-12-25,2016,True,Christmas Day
14396,Minnesota,2,2016-12-25 01:56:57,45.880870,-95.484880,Sunday,2016-12-25,2016,True,Christmas Day
14397,Texas,3,2016-12-25 06:58:52,32.760425,-97.240582,Sunday,2016-12-25,2016,True,Christmas Day


#### Now our data is all clean and ready for visualizations. We can use these various datasets we have integrated to extract different type of information